In [18]:
from google.colab import userdata
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [19]:
!pip install --quiet transformers datasets evaluate rouge-score py7zr

In [20]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, BartTokenizer, BartForConditionalGeneration
from datasets import load_dataset
import evaluate

In [21]:
# Import necessary modules
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
import evaluate

# Load the ROUGE metric
rouge = evaluate.load('rouge')

# Function to load LLM
def load_llm(model="gemini-1.5-pro", max_tokens=100):
    if model == "gemini-1.5-pro":
        llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-pro",
            temperature=0,
            max_tokens=max_tokens,
            timeout=None,
            max_retries=2)
        return llm
    elif model == "gemini-1.5-flash":
        llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash",
            temperature=0,
            max_tokens=max_tokens,
            timeout=None,
            max_retries=2)
        return llm
    else:
        raise ValueError("Invalid model name")

# Function to get prompt template
def get_prompt_template():
    # Define prompt
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Write a concise summary of the following in exactly {num_words} words. Do not exceed {num_words} words. Stick to the word count strictly:\n\n",
            ),
            ("human", "{context}")
        ]
    )
    return prompt

# Function to summarize text
def summarize_text(text, num_words=50, model="gemini-1.5-pro"):
    # Load LLM
    llm = load_llm(model)

    # Get Prompt Template
    prompt = get_prompt_template()

    # Instantiate chain
    chain = prompt | llm

    # Invoke chain
    result = chain.invoke({
        "context": text,
        "num_words": num_words
    })

    # Return result
    return result.content

# Function to calculate ROUGE score
def compute_rouge(predictions, references):
    return rouge.compute(predictions=predictions, references=references, use_stemmer=True)

# Example text for summarization
text = """
Artificial Intelligence (AI) is revolutionizing multiple industries and transforming the way we live and work.
AI technologies, including machine learning, natural language processing, and robotics, are being integrated into applications like healthcare, finance, and transportation.
In healthcare, AI is improving diagnostics and treatment, making processes faster and more accurate.
In finance, it helps detect fraudulent transactions and manage risks effectively.
AI-powered self-driving cars are reshaping transportation by reducing accidents and optimizing traffic flow.
Despite its benefits, AI poses ethical challenges, such as privacy concerns and potential job displacement.
Governments and organizations are working on policies to address these issues while fostering innovation in AI.
The future of AI looks promising, but it requires careful consideration of both its potential and its risks.
"""

# Specify the text and number of words
summary = summarize_text(text, num_words=50, model="gemini-1.5-flash")

# Print original text and summary
print(f"Original Text: {text}")
print(f"\nSummary: {summary}")
print("="*100)

# Calculate and print the number of words
original_word_count = len(text.split())
summary_word_count = len(summary.split())

print(f"\nNumber of words in the original text: {original_word_count}")
print(f"Number of words in the summary: {summary_word_count}")

# Calculate and print ROUGE scores
rouge_scores = compute_rouge([summary], [text])
print(f"\nROUGE Scores: {rouge_scores}")


Original Text: 
Artificial Intelligence (AI) is revolutionizing multiple industries and transforming the way we live and work.
AI technologies, including machine learning, natural language processing, and robotics, are being integrated into applications like healthcare, finance, and transportation.
In healthcare, AI is improving diagnostics and treatment, making processes faster and more accurate.
In finance, it helps detect fraudulent transactions and manage risks effectively.
AI-powered self-driving cars are reshaping transportation by reducing accidents and optimizing traffic flow.
Despite its benefits, AI poses ethical challenges, such as privacy concerns and potential job displacement.
Governments and organizations are working on policies to address these issues while fostering innovation in AI.
The future of AI looks promising, but it requires careful consideration of both its potential and its risks.


Summary: AI rapidly transforms healthcare, finance, and transportation throug